In [ ]:
# Install TensorFlow and other dependencies (if not already installed)
!pip install tensorflow matplotlib


#  Import Required Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import os


 # Data Preprocessing

In [ ]:

# Define dataset paths
train_dir = r'C:\xampp\htdocs\SLIIT\ENT-disease-detection-dl-models\cholesteatoma_detection_and_classification\endoscopy_image_validator\dataset\train'
val_dir = r'C:\xampp\htdocs\SLIIT\ENT-disease-detection-dl-models\cholesteatoma_detection_and_classification\endoscopy_image_validator\dataset\validation'


# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data preprocessing for validation
val_datagen = ImageDataGenerator(rescale=1./255)

# Load data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # ResNet50 input size
    batch_size=32,
    class_mode='binary'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


# Load Pre-trained ResNet50

In [ ]:
# Load ResNet50 model with ImageNet weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = layers.Flatten()(base_model.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation='sigmoid')(x)  # Binary classification

# Create the full model
model = models.Model(inputs=base_model.input, outputs=output)


# Compile the Model

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


# Train the Model

In [ ]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=[early_stopping]
)


# Evaluate the Model

In [ ]:
# Plot training history
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(val_data)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")


# Save the Model

In [ ]:
# Save the trained model
model.save('ResNet50_endoscopy_image_validator.h5')
print("Model saved as ResNet50_endoscopy_image_validator.h5")


# Load and Test the Model

In [ ]:
# Load the trained model
loaded_model = tf.keras.models.load_model('ResNet50_endoscopy_image_validator.h5')

# Test on new images
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = loaded_model.predict(img_array)
    return "Valid" if prediction[0][0] > 0.5 else "Invalid"

# Example usage
test_image_path = '/path/to/test/image.jpg'  # Replace with test image path
print(f"Prediction: {predict_image(test_image_path)}")
